In [46]:
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage import io, color
import cv2
import os
from sklearn.decomposition import PCA

# Function definitions

In [65]:
# Based on tutorial: https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
def make_square(img, desired_size=256, fill_color=[255, 255, 255]):
    if img.dtype != np.uint8:
        print(f'Converting to uint8...')
        img = (255*img).astype(np.uint8)
        
    scale_factor = desired_size/max(img.shape[0], img.shape[1])
    resized = cv2.resize(img, (int(scale_factor*img.shape[1]), int(scale_factor*img.shape[0])))
    new_size = resized.shape
    
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    out = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=fill_color)
    return out

# use float for thresholding, but return uint8 image
def binarize(img, threshold=0.5, invert=True):
    if img.dtype == np.uint8:
        img = img/255.0 # convert to float64

    # Convert to grayscale
    if len(img.shape) >= 3:
        img = color.rgb2gray(img)
    
    # Threshold
    out = np.zeros_like(img)
    if invert: # detect dark characters
        mask = img < threshold
    else: # detect light characters
        mask = img > threshold
    out[mask] = 1
    return (255*out).astype(np.uint8)

def preprocess(img, desired_size=256, fill_color=[255, 255, 255], threshold=0.5, invert=True):
    img_square = make_square(img, desired_size=desired_size, fill_color=fill_color)
    img_bin = binarize(img_square, threshold=threshold, invert=invert)
    return img_bin

def contour_analysis(img, n=2, trim_points=10, bins=20, verbose=False):
    # Find all contours
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    if verbose:
        print(f'# of contours: {len(contours)}')
        print('# of points in each contour:')
        for cnt in contours:
            print(f'\t{len(cnt)} points')

    # Remove contours with too few points
    contours_trimmed = [cnt for cnt in contours if len(cnt) > trim_points]
    if verbose:
        print(f'Trimming contours with fewer than {trim_points} points...')
        print(f'# of remaining contours: {len(contours_trimmed)}')
        for cnt in contours_trimmed:
            print(f'\t{len(cnt)} points')

    # Create dashed contours by keeping every nth point
    assert(n>=2)
    contours_dashed = [cnt[1::n] for cnt in contours_trimmed]
    if verbose:
        print(f'Taking every {n}th point to get dashed contour...')
        for cnt in contours_dashed:
            print(f'\t{len(cnt)} points')
            
    # Find angles between adjacent points in the contour
    thetaseq = []
    for i, cnt in enumerate(contours_dashed):
        for j, point in enumerate(cnt):
            if j == 0:
                prevx, prevy = point[0]
            else:
                x, y = point[0]
                thetaseq.append(np.arctan2(y-prevy, x-prevx))
                prevx = x
                prevy = y
    
    if verbose:
        print(f'# thetas: {len(thetaseq)}')
        print(f'max theta: {max(thetaseq)}')
        print(f'min theta: {min(thetaseq)}')

    hist =  np.histogram(thetaseq, bins=bins, range=(-np.pi, np.pi))
    return hist

def plot_angles(hist, ax=None, title='Distribution of angles'):
    r, theta = hist
    r = np.append(r, r[0]) # append 0th element to end cuz -pi and pi are the same angle

    if ax is None:
        fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(4, 4), dpi=300)

    ax.plot(theta, r)
    ax.grid(True)

    ax.set_title(title, va='bottom')
    plt.tight_layout()
    
def plot_mu_and_K(X, title='', filename='mu+K.png'):
    mu = np.mean(X, axis=1)
    K = np.cov(X)
    bins = len(mu)
    theta = np.linspace(-np.pi, np.pi, bins+1)
 
    fig = plt.figure(figsize=(8, 4))
    ax1 = plt.subplot(121, projection='polar')
    ax2 = plt.subplot(122)
    plot_angles([mu, theta], ax=ax1, title='Mean vector μ')
    fig.colorbar(ax2.imshow(K))
    ax2.set_title('Covariance matrix K')
    plt.suptitle(title, fontsize='xx-large')
    plt.tight_layout()
    plt.savefig(filename)

def analyze_images_in_path(path='', bins=20, verbose=False):
    filenames = os.listdir(path)
    filenames.sort()
    if verbose:
        print(filenames)
    
    X = np.zeros((bins, len(filenames)))
    for i, filename in enumerate(filenames):
        img = io.imread(path + filename)
        counts, theta = contour_analysis(preprocess(img), bins=bins)
        # divide counts by total counts to get Probability Mass Function (PMF)
        probs = counts / counts.sum()
        X[:, i] = probs

    return X

# Mi Fu's *Poem Written In a Boat on the Wu River*

In [66]:
path = 'Extracted Characters/Mi Fu - Poem Written in a Boat on the Wu River/'
filenames = os.listdir(path)
filenames.sort()
print(filenames)

['001.png', '002.png', '003.png', '004.png', '005.png', '006.png', '007.png', '008.png', '009.png', '010.png', '011.png', '012.png', '013.png', '014.png', '015.png', '016.png', '017.png', '018.png', '019.png', '020.png', '021.png', '022.png', '023.png', '024.png', '025.png', '026.png', '027.png', '028.png', '029.png', '030.png', '031.png', '032.png', '033.png', '034.png', '035.png', '036.png', '037.png', '038.png', '039.png', '040.png', '041.png', '042.png', '043.png', '044.png', '045.png', '046.png', '047.png', '048.png', '049.png', '050.png', '051.png', '052.png', '053.png', '054.png', '055.png', '056.png', '057.png', '058.png', '059.png', '060.png', '061.png', '062.png', '063.png', '064.png', '065.png', '066.png', '067.png', '068.png', '069.png', '070.png', '071.png', '072.png', '073.png', '074.png', '075.png', '076.png', '077.png', '078.png', '079.png', '080.png', '081.png', '082.png', '083.png', '084.png', '085.png', '086.png', '087.png', '088.png', '089.png', '090.png', '091.png'

In [69]:
X = analyze_images_in_path(path)
plot_mu_and_K(X, title=f'Wu River ({X.shape[1]} images)', filename='Test/mu+K_WuRiver.png')

# PCA

In [82]:
pca = PCA(n_components=2)
pca.fit(np.transpose(X))
print(f'Explained variance ratio: {pca.explained_variance_ratio_}')
print(f'Singular values: {pca.singular_values_}')
components = pca.fit_transform(np.transpose(X))
print(f'# of datapoints: {len(components)}')
print(f'Principal Components of each datapoint:\n{components}')

plt.figure(figsize=(4, 4))
plt.scatter(components[:, 0], components[:, 1], label='Wu River')
plt.title('PCA')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend()
plt.savefig('Test/PCA_WuRiver.png')

Explained variance ratio: [0.46376558 0.18567444]
Singular values: [0.64595277 0.40872158]
# of datapoints: 91
Principal Components of each datapoint:
[[ 0.00036497 -0.04055316]
 [-0.04857066 -0.07094967]
 [ 0.06122655  0.04654032]
 [-0.01613446 -0.00603782]
 [ 0.02814406 -0.01462888]
 [ 0.00671194 -0.00175597]
 [ 0.01776733  0.03306314]
 [ 0.03402     0.00555434]
 [-0.03777442 -0.049431  ]
 [-0.05745886 -0.04579842]
 [-0.00853481 -0.04679862]
 [-0.01135655 -0.00698931]
 [-0.02071141  0.00842589]
 [ 0.01274043 -0.0583408 ]
 [ 0.08382157  0.03102172]
 [-0.02297599  0.01905669]
 [ 0.10739135  0.03128132]
 [-0.05256259  0.00947863]
 [ 0.00198506 -0.08034348]
 [-0.04629028  0.01712935]
 [-0.02251515 -0.0095294 ]
 [-0.00461233 -0.00882975]
 [ 0.01980275  0.02980875]
 [-0.02633659 -0.0312785 ]
 [ 0.00528308  0.02810656]
 [ 0.01261909 -0.02890035]
 [-0.02613037  0.00528459]
 [ 0.06926955 -0.00316414]
 [-0.04079143 -0.00750503]
 [-0.02019476 -0.04851468]
 [-0.02102922  0.00042207]
 [-0.0328246

# Emperor Huizong

In [83]:
path_huizong = 'Extracted Characters/Emperor Huizong - Finches and bamboo/'
X_huizong = analyze_images_in_path(path_huizong)
plot_mu_and_K(X_huizong, title=f'Huizong ({X_huizong.shape[1]} images)', filename='Test/mu+K_Huizong.png')

In [84]:
pca = PCA(n_components=2)
pca.fit(np.transpose(X_huizong))
print(f'Explained variance ratio: {pca.explained_variance_ratio_}')
print(f'Singular values: {pca.singular_values_}')
components = pca.fit_transform(np.transpose(X_huizong))
print(f'# of datapoints: {len(components)}')
print(f'Principal Components of each datapoint:\n{components}')

plt.figure(figsize=(4, 4))
plt.scatter(components[:, 0], components[:, 1], label='Huizong')
plt.title('PCA')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend()
plt.savefig('Test/PCA_Huizong.png')

Explained variance ratio: [0.53135269 0.15757212]
Singular values: [0.50312026 0.27398075]
# of datapoints: 63
Principal Components of each datapoint:
[[-2.16313251e-03 -4.25746873e-02]
 [ 8.57089448e-02  1.40339353e-02]
 [-2.77658039e-04 -3.65770779e-02]
 [-1.27794456e-01 -7.98409233e-03]
 [ 1.07393757e-01 -2.80251417e-02]
 [-2.24144291e-02  1.00944753e-02]
 [-2.48369017e-02 -3.93555482e-02]
 [-1.10309812e-01  1.60847233e-02]
 [-3.75527438e-02  5.48116811e-03]
 [ 3.65860470e-02 -2.08294503e-02]
 [ 8.33063050e-03  3.68827932e-03]
 [ 2.43327057e-02  4.01623480e-02]
 [-6.30493270e-02  2.49007855e-03]
 [-4.33027634e-02  3.31282796e-02]
 [ 2.29656044e-02  3.74667220e-02]
 [-4.41109453e-02 -1.26432794e-02]
 [ 1.89401046e-01 -6.69006215e-02]
 [-5.87827426e-02  7.85313554e-02]
 [ 1.01260225e-01  1.84081055e-02]
 [ 1.94304092e-02 -1.21097606e-02]
 [-5.06148221e-02 -3.15385971e-02]
 [ 5.08185361e-02  1.60215374e-02]
 [-1.51775337e-02 -2.87712629e-02]
 [-1.10392628e-02  4.48198762e-02]
 [ 5.1918

In [80]:
np.mean(X, axis=1) - np.mean(X_huizong, axis=1)

array([ 0.        , -0.01189713, -0.0015365 ,  0.00282308,  0.        ,
        0.01359497,  0.0182712 ,  0.00675491, -0.00955608,  0.        ,
       -0.02040361, -0.00902268, -0.00562586,  0.00024467,  0.        ,
        0.02281982,  0.02439142, -0.00315011, -0.01308333, -0.01462477])

# PCA Comparison